# 1. Decision Tree (의사결정나무)

### 1. Decision Tree의 장단점

1) 장점
 - 결과 해석의 용이성
 - 비모수적 모델 : 통계적 가정으로부터 자유로움
 - Scaling이 필요하지 않음 (서수 기반)
 - 이상치에 민감하지 않음
 - 변수 간 상호작용 고려 가능
 - 분류, 회귀 모두 가능
 

2) 단점
 - 불안정성 : 데이터 크기가 작을 경우 불안정성이 높아짐
 - Overfitting
 - 선형관계 파악 미흡
 - 비연속성 : 연속형 변수 구간화하여 처리. 분리 경계점 근처에서 오류 발생 가능성 있음.
 - 높은 Computational Cost : 각 조합의 결과를 모두 고려할 경우
 
### 2. Decision Tree 분할 기준 (Impurity, Feature Importance)

1) Classification
 - Gini Index
 - Entropy (Shannon's Entropy 등)
 - Information Gain Ratio
 
2) Regression
 - MSE (Mean Square Error)
 - MAE (Mean Absolute Error)
 - Reduction in Possion Deviance
 - Friedman's MSE

### 3. Decision Tree의 모델 구분 기준

1) 분할(평가지표) 기준 (분산이 어떻게 계산되는가?)  
2) 종속변수 기준 (Classification / Regression)
3) 과적합 방지 기준
4) 불완전 데이터 적용 가능 여부

### 4. Decision Tree Algorithm 종류

#### Gini Index 기반
1) CART (Classification And Regression Tree) : 분류/회귀 모두 가능. 이진 분할

#### Entropy 기반
2) ID3 (Iterative Dichotomizer) : Shannon Entropy 사용 (즉, Information Gain 사용). 분류모델만 가능.
3) C4.5 : Information Gain Ratio 사용. 불완전 데이터를 처리할 수 있으며 (결측치 포함 데이터 처리 가능), Pruning이 가능하여 Overfitting 방지 가능.
4) C5.0 : C4.5의 발전된 형태. 메모리 효율성 등 성능이 향상됨.

#### 기타
5) CHAID (Chi-Square Automatic Interaction Detection) : Chai-Square Statistics (범주형), F-statistics (연속형) 사용. 다지분리 (Multiway Split) 수행.

### 4. 참고 자료
https://heytech.tistory.com/145
https://yschoi.pusan.ac.kr/sites/yschoi/download/DataMining/2-5.htm
http://ducj3.iptime.org/decisiontree/
https://tyami.github.io/machine%20learning/decision-tree-3-c4_5/

In [1]:
# 기본 설정
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn as sk
import statsmodels.api as sm

In [2]:
# matplotlib 사용 시 한글 깨짐 문제 해결
from matplotlib import font_manager, rc

font_path = "C:/Windows/Fonts/NGULIM.ttf"
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font)

In [3]:
# 경고 메시지 비활성화
import warnings

warnings.filterwarnings("ignore")

In [4]:
import sys

print(sys.executable)
print(sys.path)

D:\Anaconda3\python.exe
['C:\\Users\\lsc\\Desktop\\ADP', 'D:\\Anaconda3\\python310.zip', 'D:\\Anaconda3\\DLLs', 'D:\\Anaconda3\\lib', 'D:\\Anaconda3', '', 'D:\\Anaconda3\\lib\\site-packages', 'D:\\Anaconda3\\lib\\site-packages\\win32', 'D:\\Anaconda3\\lib\\site-packages\\win32\\lib', 'D:\\Anaconda3\\lib\\site-packages\\Pythonwin']


In [5]:
sys.path.append('D:/Anaconda3/Lib/site-packages/Graphviz/bin')

In [6]:
sys.path[-1] = sys.path[-1].replace("/", "\\")

In [7]:
print(sys.path)

['C:\\Users\\lsc\\Desktop\\ADP', 'D:\\Anaconda3\\python310.zip', 'D:\\Anaconda3\\DLLs', 'D:\\Anaconda3\\lib', 'D:\\Anaconda3', '', 'D:\\Anaconda3\\lib\\site-packages', 'D:\\Anaconda3\\lib\\site-packages\\win32', 'D:\\Anaconda3\\lib\\site-packages\\win32\\lib', 'D:\\Anaconda3\\lib\\site-packages\\Pythonwin', 'D:\\Anaconda3\\Lib\\site-packages\\Graphviz\\bin']


In [8]:
from sklearn import tree
from sklearn.datasets import load_iris
from os import system
import graphviz
# system("pip install graphviz")
# system("pip show graphviz")
  # Colab에는 이미 설치되어 있음.

iris = load_iris()
print("iris dataset attributes : ", dir(iris))

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
iris_x = pd.DataFrame(iris.data, columns = iris.feature_names)
iris_x.columns = [colnames.replace(" (cm)", "") for colnames in iris_x.columns]
iris_x.head(10)

In [ ]:
iris_y = pd.DataFrame(iris.target, columns = ["species"])
iris_y.head(10)

In [ ]:
iris_x.info()

In [ ]:
iris_y.info()

In [ ]:
iris.target_names
  # Species. 분석 종료 후 범주형으로 변경해준다.

In [ ]:
print("X 결측치 : ", iris_x.isnull().sum().sum(), "개")
print("y 결측치 : ", iris_y.isnull().sum().sum(), "개")
  # 결측치가 없는 예쁜 데이터이다.

In [ ]:
iris_y.value_counts()
  # Balanced Data이다.

## 1-1. Decision Tree Classifier : 의사결정나무 분류기 (CART)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf2 = DecisionTreeClassifier(criterion = "entropy")
clf2.fit(iris_x, iris_y)

In [ ]:
clf3 = DecisionTreeClassifier()
    # Criterion (Information Gain 방식) 기본값 : Gini Index
clf3.fit(iris_x, iris_y)

In [ ]:
# 의사결정나무 분류기 시각화 1 (Entropy)
import graphviz
    # Graphviz : Tree를 도식화하는 라이브러리.

dot_data = tree.export_graphviz(clf2, out_file = None
                                     , feature_names = iris.feature_names
                                     , class_names = iris.target_names
                                     , filled = True  # 색칠 여부
                                     , rounded = True  # 반올림 여부
                                     , special_characters = True  # 특수문자 사용 여부
)

graph = graphviz.Source(dot_data)              
graph

In [ ]:
# 의사결정나무 분류기 시각화 2 (Gini Index)

dot_data_2 = tree.export_graphviz(clf3, out_file = None
                                     , feature_names = iris.feature_names
                                     , class_names = iris.target_names
                                     , filled = True  # 색칠 여부
                                     , rounded = True  # 반올림 여부
                                     , special_characters = True  # 특수문자 사용 여부
)

graph_2 = graphviz.Source(dot_data_2)              
graph_2

### 1-1-1. Impurity (불순도) 측정

Tree의 분지(Split)를 결정하는 수단

Class가 잘 분류되어 있는 경우 불순도가 낮음.  
즉, 불순도 감소량이 최대가 되는 방향으로 분지하게 된다.

$$ I(T') = Weighted \; Impurity \; after \; split $$

$$ \Delta I(s, t) = I(T) - I(T') $$

$$ argmax \Delta I(s, t) $$


#### 1) Gini Index

 - 노드 t의 지니 지수 : $$ Gini(t) = 1 - \Sigma_{j}^{J}[p(j | t)]^2$$

 - Split (파티션)의 지니 지수 : $$ Gini_{split} = \Sigma_{i}^k {n_i \over n} Gini(i)$$

$$ n_i : 자식 \; 노드의 \; 크기 $$
$$ n : 부모 \; 노드의 \; 크기 $$

Tree의 Gini Index는 Terminal Node들의 Gini Index 값의 Weighted Sum이다.


#### 2) Entropy

$$ Entropy(t) = -\Sigma_{j}^{J} p(j | t) log_2 p(j|t)$$


#### 3) Information Gain

$$ IG = Base \; Entropy - New \; Entropy $$
$$ = \Delta I(s, t) = I(T) - I(T')$$

불순도 감소량과 같다. 즉, 불순도 감소량을 최대화하는 것은 정보 이득을 최대화하는 것과 일맥상통한다.

 
#### 4) IG Ratio (Information Gain Ratio)

그럼에도 불구하고, Information Gain 극대화에만 초점을 두면 안된다.

Information Gain이 극도로 커진다는 것은 Tree가 끝없이 커져 전체 Entropy가 0이 된다는 것을 의미하며, Train Data에 대한 Overfitting 문제가 생기게 되기 때문이다.

이를 보완하는 개념 중 하나가 IG Ratio이다.


$$ IGR = {IG \over {New \; Entropy}} = {(Base \; Entropy \, - \, New \; Entropy)  \over {New \; Entropy}} $$

IG Ratio는 Split이 많이 된 Tree에 대하여 Disadvantage를 줌으로써 Overfitting을 방지한다.

### 1-1-2. Overfitting 방지하기 : Pruning (가지치기)


사실, 위의 두 모델은 Overfitting된 것이다.

Tree의 크기에 따라 모델의 성능이 좌우될 수 있다.

 - 크기가 너무 크면 Overfitting
 - 크기가 너무 작으면 Underfitting

Overfitting, Underfitting을 방지하기 위하여 다음과 같은 방법을 사용할 수 있다.

1) Information Gain 최솟값 설정

2) 가지의 개수 제한

3) 나무의 깊이 (Depth) 제한

Pruning에는 두 종류가 있다.

#### 1-1-2-1. Pre-Pruning (Early Stopping)

Grid Search 등의 Hyperparameter Tuning 기법 등을 통하여 나무의 깊이와 가지의 개수만 조정해주면 된다.

#### 1-1-2-2. Post-Pruning
학습 후 Pruning을 진행한다.

 - Cost-Complexity Pruning (CCP)이 가장 대중적
 - Error를 최소화하는 Alpha를 찾는다.
 
1. Cost - Complexity Pruning (CCP)
 
$$ C_{\alpha}(T_k) = \sum_{t = 1}^{|T_k|} N_t i(T_k) + \alpha |T_k| \quad \; (k = 0, 1, ... , K-1, K)$$

$$ T^{*} = arg\min_{T_k} C_{\alpha}(T_k) $$

$$ T_K : 최대 \; 크기의 \; Tree $$
$$ T_K > T_{K-1} > ... > T_0  : 연속적으로 \; Pruning을 \; 진행하였을 \; 때의 \; Tree의 \; 집합 $$
$$ |T_k| : T_k의 \; Terminal \; Node \; 수$$

가장 오른쪽 항 ($ \alpha |T_k| $)은 나무의 크기에 따른 Penalty의 역할을 한다.

식을 간소화하여 살펴보자.

$$ ① \; R_\alpha (t) = R(t) + \alpha \quad (노드 \; t의 \; Impurity ; \; |T_t| = 1) $$
$$ ② \; R_\alpha (T_t) = R(T_t) + \alpha|T_t| \quad (노드 \; t가 \; Terminal \; Node인 \; Tree \; T_t의 \; Impurity)$$

$T_t$는 t보다 세분화된 모델이기 때문에, 일반적으로 $ R(T_t) < R(t)$ 이다.

그러나, 모델이 지나치게 클 경우 Overfitting 문제가 발생하여 $ R(T_t) > R(t) $가 될 수 있으므로 크기에 적절히 penalty를 줄 수 있는 $ \alpha $를 찾아야 한다.

$ |T_t| >= 1 $이므로, $ \alpha $가 커지다보면 $ R(T_t) = R(t) $가 되는 지점인 $ \alpha^{*} $를 찾을 수 있다. 

이 때, $ \alpha > \alpha^{*} $가 되면 $ R(T_t) > R(t) $, 즉 Node $ t $에서 Split을 하였을 때 오히려 성능 저하가 생기게 되어 Pruning이 필요하게 된다. (분지 여부에 따른 Impurity의 차이가 없는 부분을 찾아 Pruning을 한다는 것이다.)

$$ R_{\alpha^{*}} (t) = R_{\alpha^{*}} (T_t) $$

$$ \alpha^{*} = \frac{R(t) - R(T_t)}{|T_t|} $$

이를 만족하는 $ \alpha^{*} $에 대응하는 노드를 Weakest Link라 하며, CCP는 Weakest Link Cut이라고도 불린다.

CCP의 장점
 - Impurity를 최대한 유지하면서도 과하지 않게 Pruning → Stable Model

CCP의 단점 
 - 복잡한 알고리즘
 - 모든 Subtree를 고려하지 못함 → Local Minimum에 빠질 수 있음

2. Reduced - Error Pruning (REP)

Validation Set을 이용하여 모든 중간 마디들(Branch Node)에 대하여

split 전과 후의 Impurity 차이를 비교하여 Pruning을 수행한다.

REP의 장점
 - 직관적인 알고리즘
 
REP의 단점
 - Validation Set의 크기가 작으면 OverPruning 가능성이 있음

3. Rule Post-Pruning

In [ ]:
clf4 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 2)
clf4.fit(iris_x, iris_y)

In [ ]:
dot_data_3 = tree.export_graphviz(clf4
                                 , out_file = None
                                 , feature_names = iris.feature_names
                                 , class_names = iris.target_names
                                 , filled = True
                                 , rounded = True
                                 , special_characters = True
                                )

graph_3 = graphviz.Source(dot_data_3)
graph_3

In [ ]:
tree_attributes = [item for item in dir(DecisionTreeClassifier) if item.startswith('_') == False]
tree_attributes

# 다른 코드로는...
## tree_attributes = list(filter(lambda x : x.startswith("_") == False, dir(DecisionTreeClassifier)))

In [ ]:
clf4.__dict__
    # __dict__ : 객체의 속성 정보 확인 가능

### 1-1-3. 정확도 비교 : Confusion Matrix 활용

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# 첫번째 나무 (Entropy)
confusion_matrix(iris.target, clf2.predict(iris.data))

In [ ]:
# 두번째 나무 (Gini Index)
confusion_matrix(iris.target, clf3.predict(iris.data))

In [ ]:
# 세번째 나무 (Entropy & Pruning)
confusion_matrix(iris.target, clf4.predict(iris.data))

# 가지치기를 한 Tree의 정확도가 가장 떨어진다.

### 1-1-4. 데이터 분할 : Train Set / Validation Set / Test Set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 이 데이터는 크기가 비교적 작은 편이기 때문에, Validation Set을 따로 두지 않고 진행한다.

iris_x_train, iris_x_test, iris_y_train, iris_y_test = train_test_split(iris.data, iris.target
                                                                        , test_size = 0.3
                                                                        , random_state = 123
                                                                        , stratify = iris.target
                                                                       )

    # stratify : 층화추출 적용
    # Iris 데이터는 완벽한 Balanced Data이기 때문에, 무작위 추출 시 Imbalanced Data로 변할 위험이 있다.

In [ ]:
# Training Set으로 다시 모델링
clf5 = tree.DecisionTreeClassifier(criterion = 'entropy')
clf5.fit(iris_x_train, iris_y_train)

In [ ]:
confusion_matrix(iris_y_test, clf5.predict(iris_x_test))
    # 가지치기 없이도 오분류율이 높아짐.
    # 데이터 분할 시 Training Set과 Test Set의 특성이 조금씩 달라지기 때문이다.

## 1-2. Decision Regression Tree : 의사결정회귀나무 (CART)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
rng = np.random.RandomState(123)
    # np.random.RandomState : 난수 생성기

x = np.sort(5 * rng.rand(80, 1), axis = 0)
    # np.random.RandomState.rand(m, n) : 균등분포 U(0, 1)에서 난수 Array (m, n) 생성
    # np.random.randn(m, n) : Gaussian Normal Distribution N(0, 1)에서 난수 Array (m, n) 생성
y = np.sin(x).ravel()
y[::5] += 3 * (0.5 - rng.rand(16))

In [ ]:
x_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]

In [ ]:
# Depth가 다른 두 회귀트리 생성 & 적합
reg1 = tree.DecisionTreeRegressor(max_depth = 2)
    # 분리 기준 : MSE, MAE, Poisson Deviance (포아송 편차 감소도), Friedman's MSE 
reg2 = tree.DecisionTreeRegressor(max_depth = 5)

reg1.fit(x, y)
reg2.fit(x, y)

In [ ]:
y1 = reg1.predict(x_test)
y2 = reg2.predict(x_test)

In [ ]:
# 도식화
plt.figure()

plt.scatter(x, y
           , s = 20
           , edgecolor = "black"
           , c = "pink"
           , label = "Data")
plt.plot(x_test, y1
        , color = "cornflowerblue"
        , label = "Model 1 : 최대 깊이 2"
        , linewidth = 2)

plt.plot(x_test, y2
        , color = "yellowgreen"
        , label = "Model 2 : 최대 깊이 5"
        , linewidth = 2)

plt.xlabel("Data")
plt.ylabel("Target")
plt.title("의사결정나무 회귀트리")
plt.legend()

plt.show()

#### max_depth = 5인 나무가 이상치의 영향을 더 많이 받았음.